## Preprocessing

In [48]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [49]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])


In [50]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [51]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [52]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [53]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"


In [54]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
print(class_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [55]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [56]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [57]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

from tensorflow.keras.layers import Dropout

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=X_train.shape[1]))
nn.add(Dropout(0.2))  # Drop 20% of neurons

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(Dropout(0.2))  # Drop 20% of neurons

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))
nn.add(Dropout(0.2))  # Drop 20% of neurons

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=100)

Epoch 1/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - accuracy: 0.6946 - loss: 0.6052
Epoch 2/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.7275 - loss: 0.5569
Epoch 3/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - accuracy: 0.7301 - loss: 0.5486
Epoch 4/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.7323 - loss: 0.5502
Epoch 5/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - accuracy: 0.7346 - loss: 0.5466
Epoch 6/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.7321 - loss: 0.5466
Epoch 7/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.7353 - loss: 0.5453
Epoch 8/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.7303 - loss: 0.5472
Epoch 9/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.7310 - loss: 0.5479
Epoch 10/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.7302 - loss: 0.5498
Epoch 11/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.7336 - loss: 0.5471
Epoch 12/50
275/275 ━━━━━━━━━━

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7206 - loss: 0.5650
Loss: 0.5649600625038147, Accuracy: 0.7205539345741272


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupopt1.h5")